# CNN迁移学习

## 环境导入

参数区

In [1]:
DATASET_PATH_ROOT = '/data/DataSets/TWITTER_IMG_SENT_2015/dataset/'
OUT_PATH_ROOT = '/data/Models/TWITTER_SENT_2015/'
OUT_LOG_PATH = OUT_PATH_ROOT + 'logs/'
BATCH_SIZE = 512
TOTAL_EPOCH = 100

In [2]:
import os
## 导入 Inceptionv3 模型
from keras.applications.inception_v3 import InceptionV3, preprocess_input

## 导入建立神经网络的基本模块
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D

from keras.optimizers import *
from keras.losses import categorical_crossentropy

## 导入数据增强模块
import cv2
from keras_preprocessing.image import ImageDataGenerator

# 可视化
# from keras.utils import plot_model
# from keras_visualizer import visualizer
# from IPython.display import Image, SVG, display
from keras.callbacks import TensorBoard
import datetime


2023-01-06 21:26:53.328618: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


## 数据准备

In [3]:

# 数据增强
train_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

#
val_datagen = ImageDataGenerator(
        preprocessing_function=preprocess_input,
        # rescale=1. / 255,
        rotation_range=30,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
)

# 数据输入 #Inception V3规定大小
train_generator = train_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}train', target_size=(299, 299), batch_size=BATCH_SIZE)
val_generator = val_datagen.flow_from_directory(directory=f'{DATASET_PATH_ROOT}validation', target_size=(299, 299), batch_size=BATCH_SIZE)

Found 617 images belonging to 2 classes.
Found 176 images belonging to 2 classes.


## 模型设定

In [4]:
# 构建基础模型
base_model = InceptionV3(weights='imagenet', include_top=False)  #去掉最后一层

# 增加新的输出层
x = base_model.output
x = GlobalAveragePooling2D()(x)  # 添加全局平均池化层 将 MxNxC 的张量转换成 1xC 张量，C是通道数
x = Dense(1024, activation='relu')(x)  # 添加一个全连接层
predictions = Dense(2, activation='softmax')(x)  # 自定义自己的分类器，这是一个2分类的分类器
model = Model(inputs=base_model.input, outputs=predictions)  # 构建我们需要训练的完整模型

2023-01-06 21:26:55.243252: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 21:26:55.251679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 21:26:55.253212: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-06 21:26:55.255151: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFl

## 迁移学习

In [5]:
# 输出日志
log_dir = OUT_LOG_PATH + 'fit/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
os.environ['TENSORBOARD_BINARY'] = '/usr/local/miniconda3/envs/TensorFlow/bin/tensorboard'

# @formatter:off
%load_ext tensorboard
%tensorboard --logdir {log_dir} --port 6006 --bind_all
# @formatter:on

Launching TensorBoard...

In [ ]:
# 锁层
for layer in base_model.layers:
    layer.trainable = False

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])  # rmsprop

# 拟合模型
history_tl = model.fit_generator(generator=train_generator,
                                 steps_per_epoch=(617 // BATCH_SIZE),  #800
                                 epochs=TOTAL_EPOCH,  #2
                                 validation_data=val_generator,
                                 validation_steps=(176 // BATCH_SIZE),  #12
                                 class_weight=None,  #不要这个,
                                 callbacks=[TensorBoard(log_dir=log_dir, profile_batch=5, histogram_freq=1)]
                                 )
# 保存
model.save(f'{OUT_PATH_ROOT}Twitter2015_iv3_tl.h5')

2023-01-06 21:27:01.554440: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-01-06 21:27:01.554481: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.
2023-01-06 21:27:01.554520: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1665] Profiler found 1 GPUs
2023-01-06 21:27:01.796254: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
/tmp/ipykernel_10651/2262979340.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history_tl = model.fit_generator(generator=train_generator,
2023-01-06 21:27:01.798769: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed


Epoch 1/100


2023-01-06 21:27:27.694415: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2023-01-06 21:27:28.502350: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 34s 34s/step - loss: 0.8131 - accuracy: 0.3730
Epoch 2/100
1/1 [==============================] - 13s 13s/step - loss: 6.6307 - accuracy: 0.6758
Epoch 3/100
1/1 [==============================] - 13s 13s/step - loss: 4.1017 - accuracy: 0.6699
Epoch 4/100
1/1 [==============================] - 9s 9s/step - loss: 1.4719 - accuracy: 0.6667
Epoch 5/100


2023-01-06 21:28:27.771513: I tensorflow/core/profiler/lib/profiler_session.cc:99] Profiler session initializing.
2023-01-06 21:28:27.771553: I tensorflow/core/profiler/lib/profiler_session.cc:114] Profiler session started.


1/1 [==============================] - ETA: 0s - loss: 0.5495 - accuracy: 0.7619

2023-01-06 21:28:30.619713: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2023-01-06 21:28:30.623023: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1799] CUPTI activity buffer flushed
2023-01-06 21:28:30.664436: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:521]  GpuTracer has collected 573 callback api events and 572 activity events. 
2023-01-06 21:28:30.676577: I tensorflow/core/profiler/lib/profiler_session.cc:126] Profiler session tear down.
2023-01-06 21:28:30.687110: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: /data/Models/TWITTER_SENT_2015/logs/fit/20230106-212658/plugins/profile/2023_01_06_21_28_30

2023-01-06 21:28:30.694958: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to /data/Models/TWITTER_SENT_2015/logs/fit/20230106-212658/plugins/profile/2023_01_06_21_28_30/VM-0-11-ubuntu.trace.json.gz
2023-01-06 21:28:30.734703: I 

1/1 [==============================] - 5s 5s/step - loss: 0.5495 - accuracy: 0.7619
Epoch 6/100
1/1 [==============================] - 5s 5s/step - loss: 1.0349 - accuracy: 0.3238
Epoch 7/100
1/1 [==============================] - 5s 5s/step - loss: 1.0429 - accuracy: 0.3429
Epoch 8/100
1/1 [==============================] - 5s 5s/step - loss: 0.8627 - accuracy: 0.3905
